In [2]:
import os, json, tqdm
import pandas as pd

## Conversion: VGG Image Annotator format to COCO 
Necessary actions to modify the VIA json file before feeding it into the converting function in ```converters.ipynb```.

In [2]:
# Step: REMOVE DATA - Remove no-annotated images
path_with_notannos = '/Users/francescabianchessi/THESIS/ANNOTATING_DF2/df_shop.json'
js = json.load(open(path_with_notannos))
new_js = js.copy()
new_js['_via_img_metadata'] = {k:v for k,v in js['_via_img_metadata'].items() if int(k) <= 3083 and v['regions']}
new_js['_via_image_id_list'] = [name for name in js['_via_image_id_list'] if str(name) in new_js['_via_img_metadata'].keys()]
print(len(new_js['_via_image_id_list']))

with open('post_delete.json', 'w') as f:
    json.dump(new_js, f)

3001


In [3]:
# Step: BASE - Open a json and checks 
path_annos_train_val = '/Users/francescabianchessi/THESIS/ANNOTATING_DF2/PROVOOO.json'
j = json.load(open(path_annos_train_val)) #'_via_img_metadata', '_via_attributes', '_via_data_format_version', '_via_image_id_list'
assert list(j['_via_img_metadata'].keys()) == [str(i) for i in j['_via_image_id_list']]

for d in j['_via_img_metadata'].values(): # check all images have annos
    for r in d['regions']:
        try:
            assert r['region_attributes']['clothing']
        except: 
            print(d['filename']) # names images without annos

masks_count = []
for k, v in j['_via_img_metadata'].items():
    masks_count.append((int(k.split('.jpg')[0]), len(j['_via_img_metadata'][k]['regions'])))

ids_segmented = []
for t in masks_count:
    if t[1] > 0:
        ids_segmented.append(t[0])

assert len(ids_segmented) == len(j['_via_img_metadata'])
num_masks = sum([t[1] for t in masks_count])
num_images = len(j['_via_img_metadata'])

print(num_masks, num_images, num_masks/num_images)

1893 500 3.786


In [5]:
# Step: SPLIT TRAIN/VALIDATION
train, val = {}, {}
for i, k in enumerate(j['_via_img_metadata']):
    if i < 2500: # customize here @@@
        train[k] = j['_via_img_metadata'][k]
    else:
        val[k] = j['_via_img_metadata'][k]
print(len(train), len(val))

# TRAIN
new_js_tr = j.copy()
new_js_tr['_via_img_metadata'] = train
new_js_tr['_via_image_id_list'] = [name for name in j['_via_image_id_list'] if str(name) in train.keys()]
print(len(new_js_tr['_via_image_id_list']))

with open('train_'+str(len(train))+'.json', 'w') as f:
    json.dump(new_js_tr, f)

# VAL
new_js_v = j.copy()
new_js_v['_via_img_metadata'] = val
new_js_v['_via_image_id_list'] = [name for name in j['_via_image_id_list'] if str(name) in val.keys()]
print(len(new_js_v['_via_image_id_list']))

with open('val_'+str(len(val))+'.json', 'w') as f:
    json.dump(new_js_v, f)

2500 501
2500
501


In [4]:
# Step: GET FILENAMES - step necessary to rename images and save them in a new folder
# NB: need to run BASE step
filenames = [v['filename'] for v in j['_via_img_metadata'].values()]
assert len(filenames) == len(ids_segmented) == len(j['_via_img_metadata'].keys())

with open('names_images__.txt', 'w') as f:
    f.write('\n'.join(filenames))

In [5]:
# Step: RENAME + MOVE NEW FOLDER - Run copy_files.py + rename in json
path_new_folder = '/Users/francescabianchessi/THESIS/CODE_TESI/datasets/dlcv/dlcv_test'
check_copy = os.listdir(path_new_folder)
assert len(filenames) == len(ids_segmented) == len(j['_via_img_metadata'].keys()) == len(check_copy)

i = 0
for k,v in j['_via_img_metadata'].items():
    v['filename'] = str(i).zfill(6)+'.jpg'
    i+=1

with open('via_df2_ts_'+str(len(check_copy))+'.json', 'w') as f:
    json.dump(j, f)

# Open with VIA to check: re-save with custom project name